# 1 Preliminaries
## 1.1 Import

In [9]:
import tensorflow as tf
import keras.api.layers as layers
from keras.api.callbacks import EarlyStopping
from keras.api.losses import BinaryCrossentropy
from keras.api.metrics import BinaryAccuracy
from keras.api.models import Model
from keras.api.preprocessing import image_dataset_from_directory
from keras.api.optimizers import Adam
import pandas as pd

## 1.2 Load images

In [10]:
landmarks_ds_train = image_dataset_from_directory(
    '../data/landmark',
    labels='inferred',
    label_mode='binary',
    image_size=(512, 512),
    shuffle=True,
)

no_landmarks_ds_train = image_dataset_from_directory(
    '../data/no_landmark',
    labels='inferred',
    label_mode='binary',
    image_size=(512, 512),
    shuffle=True,
)

Found 1 files belonging to 2 classes.


# 2 Train model
## 2.1 Create model

In [11]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]

def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

In [12]:
inputs = layers.Input(shape=(512, 512, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

previous_block_activation = x

for size in [256, 512, 728]:
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # Project residual
    residual = layers.Conv2D(size, 1, strides=2, padding="same")(
        previous_block_activation
    )
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual

x = layers.SeparableConv2D(1024, 3, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(1, activation=None)(x)

model = Model(inputs=inputs, outputs=outputs)

## 2.2 Train model

In [ ]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5),
]

model.compile(
    optimizer=Adam(3e-4),
    loss=BinaryCrossentropy(from_logits=True),
    metrics=[BinaryAccuracy(name="acc")],
)

history = model.fit(
    landmarks_ds_train,
    epochs=25,
    callbacks=callbacks,
    validation_split=0.2,
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'accuracy']].plot() # plot loss and accuracy

model.save('../models/ai.keras') # save model